# Verifier Agent

In [21]:
import requests
import json
import qrcode
import uuid
import re

In [22]:
# Set the URL for the API endpoint
# Need to specify the host as host.docker.internal as the Prism Agents and the Prism Playground are running in two isolated docker environments
base_url = "http://host.docker.internal:9000"

headers = {'Content-Type': 'application/json', 'accept': 'application/json'}

## Connections

In [23]:
path = '/prism-agent/connections'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

{
  "contents": [],
  "kind": "Collection",
  "self": "/collections"
}


### Create a connection record containing the invitation
```bash
curl -X 'POST' \
	'http://localhost:8080/prism-agent/connections' \
	-H 'Content-Type: application/json' \
	-d '{
		"label": "Connect with Alice"
		}' | jq
```

In [24]:
path = '/prism-agent/connections'
data = {"label": "Connect with Alice"}
# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 201:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-201 status code")

{
  "connectionId": "d92a46e3-e34b-4f85-83e0-c90e9f5e6703",
  "createdAt": "2022-12-14T18:43:17.879280Z",
  "invitation": {
    "from": "did:peer:2.Ez6LSoWmtXjFXcaQEcjRzv7F1aLAdbQjjnXzxHnu72fchvCm5.Vz6MkvDhmbYiNUrWvbSt5qbUjdFZsLDKgBUbQKFSVYwqrzELW.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjkwMDAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0",
    "id": "d92a46e3-e34b-4f85-83e0-c90e9f5e6703",
    "invitationUrl": "https://domain.com/path?_oob=eyJpZCI6ImQ5MmE0NmUzLWUzNGItNGY4NS04M2UwLWM5MGU5ZjVlNjcwMyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNvV210WGpGWGNhUUVjalJ6djdGMWFMQWRiUWpqblh6eEhudTcyZmNodkNtNS5WejZNa3ZEaG1iWWlOVXJXdmJTdDVxYlVqZEZac0xES2dCVWJRS0ZTVll3cXJ6RUxXLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qa3dNREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBi

## Send invitation to Holder

In [25]:
invitation = json.loads(response.content.decode())
invitationUrl = invitation['invitation']['invitationUrl']
invitationUrlbase64verifier = re.sub("^.*_oob=", "", invitationUrl)
print(invitationUrlbase64verifier)
connection_id = invitation['connectionId']
%store invitationUrlbase64verifier

eyJpZCI6ImQ5MmE0NmUzLWUzNGItNGY4NS04M2UwLWM5MGU5ZjVlNjcwMyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNvV210WGpGWGNhUUVjalJ6djdGMWFMQWRiUWpqblh6eEhudTcyZmNodkNtNS5WejZNa3ZEaG1iWWlOVXJXdmJTdDVxYlVqZEZac0xES2dCVWJRS0ZTVll3cXJ6RUxXLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qa3dNREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBiZXR3ZWVuIHR3byBwZWVycyIsImFjY2VwdCI6W119fQ==
Stored 'invitationUrlbase64verifier' (str)


### Generate QR code from invitation

In [ ]:
# Link for connection invitation
input_data = invitationUrl
# Creating an instance of qrcode
qr = qrcode.QRCode(
        version=1,
        box_size=10,
        border=5)
qr.add_data(input_data)
qr.make(fit=True)
img = qr.make_image(fill='black', back_color='white')
img.save('issuer_agent_invite_QRcode.png')

from IPython.display import Image
Image(width=400, filename='./issuer_agent_invite_QRcode.png')

### Retrieving the list of connections
```bash
curl -X 'GET' 'http://localhost:8080/prism-agent/connections' | jq
```

In [29]:

path = '/prism-agent//connections/' + connection_id

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    STATE = loaded_json['state']
    print(STATE)
    

else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

{
  "connectionId": "d92a46e3-e34b-4f85-83e0-c90e9f5e6703",
  "createdAt": "2022-12-14T18:43:17Z",
  "invitation": {
    "from": "did:peer:2.Ez6LSoWmtXjFXcaQEcjRzv7F1aLAdbQjjnXzxHnu72fchvCm5.Vz6MkvDhmbYiNUrWvbSt5qbUjdFZsLDKgBUbQKFSVYwqrzELW.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjkwMDAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0",
    "id": "d92a46e3-e34b-4f85-83e0-c90e9f5e6703",
    "invitationUrl": "https://domain.com/path?_oob=eyJpZCI6ImQ5MmE0NmUzLWUzNGItNGY4NS04M2UwLWM5MGU5ZjVlNjcwMyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNvV210WGpGWGNhUUVjalJ6djdGMWFMQWRiUWpqblh6eEhudTcyZmNodkNtNS5WejZNa3ZEaG1iWWlOVXJXdmJTdDVxYlVqZEZac0xES2dCVWJRS0ZTVll3cXJ6RUxXLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qa3dNREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBiZXR3ZWV

KeyError: 'theirDid'

In [33]:
import time
from termcolor import colored,cprint

path = '/prism-agent//connections/' + connection_id

# print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'ConnectionResponseSent':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json['state']
        print(colored("ConnectionId {0} is not in active state yet. State: {1}".format(connection_id,STATE), "yellow", attrs=["bold"]))
        their_did = loaded_json['theirDid']
    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))


Current state for ConnectionId d92a46e3-e34b-4f85-83e0-c90e9f5e6703 is ConnectionResponseSent
ConnectionId: d92a46e3-e34b-4f85-83e0-c90e9f5e6703 is now active. Continue with notebook


## Executing the `Verifier` flow
---
connectionId is the holder (connectionId or did)
Replace `{CONNECTION_ID}` with the DID of the holder displayed at startup in the his Prism Agent console logs

### Initiates a Proof Request

```shell
curl -X 'POST' \
  'http://localhost:8070/prism-agent/present-proof/presentations' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "connectionId": "{CONNECTION_ID}", "proofs":[]
}'
```

In [34]:
path = '/prism-agent/present-proof/presentations'
data = {
  "connectionId": connection_id, "proofs":[]
}
# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 201:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-201 status code")
    print(response.content)

{
  "presentationId": "13d21a99-ffcd-40e9-b158-42ec2c26ccd2"
}


### Send proof request to Holder

In [35]:
presentation_id = loaded_json['presentationId']
%store presentation_id
print(presentation_id)

Stored 'presentation_id' (str)
13d21a99-ffcd-40e9-b158-42ec2c26ccd2


### Check Presentation state  PresentationReceived 
> check PresentationReceived !
```shell
curl -X 'GET' 'http://localhost:8070/prism-agent/present-proof/presentations' -H 'accept: application/json' | jq
```


In [37]:
path = '/prism-agent/present-proof/presentations'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    # print(response.content)
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=1))
    # proof_requestId = loaded_json[0]['presentationId']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

[
 {
  "presentationId": "13d21a99-ffcd-40e9-b158-42ec2c26ccd2",
  "proofs": [],
  "status": "PresentationReceived"
 }
]


### Accept PresentationReceived 
> Replace `{PRESENTATION_ID}` with the UUID of the record from the presentation records list with state PresentationReceived

```shell
curl -X 'PATCH' \
  'http://localhost:8070/prism-agent/present-proof/presentations/{PRESENTATION_ID}' \
  -H 'Content-Type: application/json' \
  -d '{"action": "presentation-accept"}' | jq
```



In [41]:
data = {"action": "presentation-accept"}

path = '/prism-agent/present-proof/presentations/' + presentation_id

# Make the request
response = requests.patch(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    print(response.content)
    # loaded_json = json.loads(response.content.decode())
    # print(json.dumps(loaded_json, indent=2))
    # proof_requestId = loaded_json[0]['presentationId']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

Error: The API returned a non-200 status code
b'{"detail":"RepositoryError(java.rmi.UnexpectedException: Invalid row count result: 0)","instance":"error-instance","status":500,"title":"error-title","type":"error-type"}'


### Check Presentation state  PresentationVerified 
> check PresentationVerified !

```shell
curl -X 'GET' 'http://localhost:8070/prism-agent/present-proof/presentations' -H 'accept: application/json' | jq
```

In [46]:
path = '/prism-agent/present-proof/presentations'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

[
  {
    "presentationId": "13d21a99-ffcd-40e9-b158-42ec2c26ccd2",
    "proofs": [],
    "status": "PresentationVerified"
  }
]


## DIDs

### Create a new unpublised DID stored in Prism Agent
```bash
curl --location --request POST 'http://localhost:8080/prism-agent/did-registrar/dids' \
  --header 'Content-Type: application/json' \
  --header 'Accept: application/json' \
  --data-raw '{
    "documentTemplate": {
      "publicKeys": [
        {
          "id": "auth0",
          "purpose": "authentication"
        }
      ],
      "services": []
    }
  }'
```

In [ ]:
path = '/prism-agent/did-registrar/dids'
data = {
    "documentTemplate": {
      "publicKeys": [
        {
          "id": "auth0",
          "purpose": "authentication"
        }
      ],
      "services": []
    }
  }

# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

### Lists all the DIDs in Prism Agent
```bash
curl --location --request GET 'http://localhost:8080/prism-agent/did-registrar/dids' \
  --header 'Accept: application/json'
```



In [ ]:
path = '/prism-agent/did-registrar/dids'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    DID_REF = loaded_json[0]['longFormDid']
    DID_REF_SHORT = loaded_json[0]['did']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

### Publishes the DID in Prism Agent to the blockchain
Replace `DID_REF` by the DID on Prism Agent that should be published
```bash
curl --location --request POST 'http://localhost:8080/prism-agent/did-registrar/dids/{DID_REF}/publications' \
--header 'Accept: application/json'
```

In [ ]:
path = '/prism-agent/did-registrar/dids/' + DID_REF + '/publications'

# Make the request
response = requests.post(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 202:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-202 status code")
    print(response.content)

### Resolves the DID document of Prism DID
Replace `DID_REF` by the DID on Prism Agent that should be resolved
```bash
curl --location --request GET 'http://localhost:8080/prism-agent/dids/{DID_REF}' \
--header 'Accept: application/json'
```

In [ ]:
path = '/prism-agent/dids/' + DID_REF

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)